# **3D point cloud of Siple Coast active subglacial lakes**

Making a LiDAR point cloud of Antarctica for an 2021 AGU poster.
Will be uploaded as a 3D model to [Sketchfab](https://sketchfab.com).

In [1]:
import cmcrameri.cm as cmc
import pandas as pd
import pygmt
import pyvista as pv

## Load ICESat-2 point cloud over Siple Coast

In [2]:
# Load Siple Coast pre-processed ATL11 point cloud data
# This 4.6GB file was processed using the atlxi_dhdt.ipynb script from
# https://github.com/weiji14/deepicedrain/pull/329/commits/9073678a2adb42fcc863afec22957c879988fa3d
df_dhdt: pd.DataFrame = pd.read_parquet(path="df_dhdt_siple_coast.parquet")

In [3]:
# Filter point cloud to those with rate of elevation change (dhdt)
# that is -0.12 m/yr < dhdt_slope > 0.12 m/yr
_df_dhdt = df_dhdt[abs(df_dhdt.dhdt_slope) > 0.12]  # 0.105
points: pd.DataFrame = (
    _df_dhdt[["x", "y", "h_corr_11", "dhdt_slope"]]
    .rename(columns=dict(h_corr_11="z"))
    .dropna()
)

# Add some vertical exaggeration (x100) to z-axis
points["z"] *= 100
points

,x,y,z,dhdt_slope
302,-802410.747040,-106989.154175,199090.625000,0.124508
304,-802517.779339,-107035.437614,199336.000000,0.121043
309,-802785.361000,-107151.145747,199977.250000,0.121403
310,-802838.876575,-107174.289345,200098.046875,0.121118
17336,-519643.152282,-235294.398466,157506.828125,-0.254864
...,...,...,...,...
47760427,149823.633547,-654345.713111,246131.156250,-0.940599
47760428,149830.391271,-654287.870761,246302.953125,-2.756242
47760429,149837.141303,-654230.020594,246527.203125,-1.920799
47760431,149850.155510,-654114.277456,246791.156250,-1.553098


## Wrap ICESat-2 data in PyVista format

In [4]:
# Create XYZ point cloud in pyvista format
cloud = pv.PolyData(var_inp=points[["x", "y", "z"]].to_numpy())

In [5]:
# Add dhdt_slope as an attribute to the XYZ point cloud
cloud.point_data["dhdt_slope"] = points.dhdt_slope
cloud

PolyData (0x7f56e5274580)
  N Cells:	9445483
  N Points:	9445483
  X Bounds:	-1.000e+06, 2.500e+05
  Y Bounds:	-1.000e+06, -1.000e+05
  Z Bounds:	-1.927e+06, 5.896e+06
  N Arrays:	1

## Render the point cloud!

In [ ]:
# Make a 3D PyVista plot of the XYZ point cloud, colored by dhdt_slope
p = pv.Plotter()
p.add_points(points=cloud, clim=[-2.5, 2.5], cmap=cmc.vik_r)
p.export_gltf(filename="siple_coast_point_cloud.gltf")
p.show(cpos="xy")

In [7]:
# https://superuser.com/questions/281573/what-are-the-best-options-to-use-when-compressing-files-using-7-zip/742034#742034
# !7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on siple_coast_point_cloud.gltf.7z siple_coast_point_cloud.gltf


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_NZ.UTF-8,Utf16=on,HugeFiles=on,64 bits,80 CPUs Intel(R) Xeon(R) Gold 6138 CPU @ 2.00GHz (50654),ASM,AES-NI)

Scanning the drive:
  0M Sca        1 file, 302259976 bytes (289 MiB)

Creating archive: siple_coast_point_cloud.gltf.7z

Items to compress: 1

      0% + siple_coast_point_cloud.glt                                    0% 1 + siple_coast_point_cloud.glt                                      1% 1 + siple_coast_point_cloud.glt                                      2% 1 + siple_coast_point_cloud.glt                                      4% 1 + siple_coast_point_cloud.glt                                      5% 1 + siple_coast_point_cloud.glt                                      6% 1 + siple_coast_point_cloud.glt                                      7% 1 + siple_coast_point_cloud.glt                                      8% 1 + siple_coast_point_cloud.glt                                   

In [ ]:
# Quick plot
# cloud.plot(cpos="xy", render_points_as_spheres=True, clim=[-2.5, 2.5], cmap=cmc.vik_r)